In [15]:
# Import necessary libraries
import dspy
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI language model
cheap_lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="low")
smart_lm = dspy.LM("openai/gpt-5", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="high")

# Define the Comedian class signature
class ComedianSignature(dspy.Signature):
    """
    Tell me a funny joke.
    """
    topic: str = dspy.InputField(description="The message to analyze")
    joke: str = dspy.OutputField(description="A funny joke about the topic")

# Define the comedian program
comedian = dspy.Predict(ComedianSignature)

# Create instances of Comedian with different LMs
with dspy.context(lm=cheap_lm):
    cheap_response = comedian(topic="programming")

with dspy.context(lm=smart_lm):
    smart_response = comedian(topic="programming")

print("Cheap LM response:")
print(cheap_response.joke)

print("\nSmart LM response:") 
print(smart_response.joke)

Cheap LM response:
Why do programmers prefer dark mode? Because light attracts bugs — and they already have enough features labeled "undocumented."

Smart LM response:
There are only two hard things in programming: cache invalidation, naming things, and off-by-one errors.


In [19]:
from typing import Literal

dspy.configure(lm=cheap_lm)

# Define the Persona class signature
class AudienceSignature(dspy.Signature):
    """
    Decide if the joke is funny.
    """
    joke: str = dspy.InputField(description="A joke to evaluate")
    profiles: list[str] = dspy.InputField(description="Profiles of the audience members")
    responses: list[Literal["hilarious", "funny", "meh", "not funny", "offensive"]] = dspy.OutputField(description="Rating of how funny the joke is from each audience member")

# Create audience evaluator class
class AudienceEvaluator(dspy.Module):
    """
    DSPy module that defines five audience personas and queries them
    to produce an aggregate funniness score for a given joke.
    """
    def __init__(self):
        super().__init__()
        # Define audience personas inside the module (single source of truth)
        self.profiles = [
            "Tech-savvy software engineer who loves clever wordplay and programming jokes",
            "Conservative grandmother who prefers clean, family-friendly humor", 
            "College liberal student who is easily offended and finds problematic undertones in most jokes",
            "Middle-aged business professional who appreciates witty observations",
            "Stand-up comedy enthusiast who has heard every classic joke"
        ]
        # Predictor that asks personas to rate the joke using chain-of-thought
        self.audience = dspy.Predict(AudienceSignature)
        # Map qualitative ratings to a numeric score
        self.rating_scores = {
            "hilarious": 5,
            "funny": 4,
            "meh": 3, 
            "not funny": 2,
            "offensive": 1
        }

    def forward(self, joke: str) -> float:
        """
        Query each persona and return average score out of 5 to 2 decimal places.
        """
        # Get ratings from all personas at once
        response = self.audience(joke=joke, profiles=self.profiles)
        
        # Calculate total score from list of ratings
        total_score = sum(self.rating_scores[rating] for rating in response.responses)
            
        # Calculate average score out of 5 to 2 decimal places
        avg_score = round(total_score / len(self.profiles), 2)
        
        # Return average score
        return avg_score

# Create evaluator instance        
audience = AudienceEvaluator()

# Example usage
joke = cheap_response.joke
score = audience(joke)
print(f"Average audience score for joke: {score}/5")

# Let's try another example
joke2 = smart_response.joke
score2 = audience(joke2)
print(f"Average audience score for joke2: {score2}/5")



Average audience score for joke: 3.8/5
Average audience score for joke2: 3.6/5
